In [5]:
%load_ext sql

In [23]:
import os
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine,inspect
from shapely.geometry import Point


In [8]:
# Access the environment variables
user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')
host = 'localhost'  # Or your database host
port = '5432'       # Default PostgreSQL port
database = 'master_geo' # DB name

# Create the connection string
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{database}'

engine = create_engine(connection_string)

In [11]:
insp = inspect(engine)
insp.get_table_names()

['spatial_ref_sys', 'pointcloud_formats', 'topology', 'layer']

In [21]:
sql = 'SELECT * FROM topology'
gdf = pd.read_sql(sql, con=engine)
gdf

,id,name,srid,precision,hasz


### Read all Facilities and convert o gdf

In [22]:
df = pd.read_csv('data/all_facilities.csv') 

,code,name,officialname,registration_number,keph_level_name,facility_type_name,facility_type_category,county,constituency,ward,...,infrastructure_categories,speciality_names,speciality_categories,is_public_visible,created,closed,is_published,id,lat,long
0,31996.0,Nyamira Teaching & Referral Hospital Private Wing,Nyamira Teaching and Referral Hospital Private...,NaN,Level 5,Secondary care hospitals,HOSPITALS,bbc8803a-7d6f-411a-96f3-5f8472b40405,b6d8c1ed-7411-4368-8ffe-36f506064458,39046919-0756-45c4-a84d-fdb584885ee5,...,[],"['Anaesthetist Nurse', 'Clerks', 'Human Resour...","['DENTAL STAFF', 'HEALTH ADMINISTRATIVE STAFFS...",True,2024-08-02T06:53:18.962471Z,False,True,accf47e7-2a44-4fd3-bbad-a711829e3927,34.929272,-0.557114
1,31999.0,Garissa Level 3 Hospital (Modika),Garissa Level 3 Hospital (Modika),NaN,Level 3,Comprehensive Health Centre,HEALTH CENTRE,4c6396a2-b23a-436a-a457-50e29a7eec6f,b27d2f78-2ed5-4831-8e41-fc8e31b02010,8f41801c-af0d-49e1-b519-79d0f9926326,...,[],"['General Medical Officers', 'Casuals', 'Pharm...","['DENTAL STAFF', 'MEDICAL OFFICERS & SPECIALIS...",True,2024-08-01T11:34:03.514866Z,False,True,55b23108-e2bd-4ceb-b58b-eb008a40ed0f,39.746100,-0.395990
2,31995.0,Equity Afia Medical Centre (Garissa),Equity Afia Medical Centre (Garissa),021173,Level 3,Medical Center,MEDICAL CENTER,4c6396a2-b23a-436a-a457-50e29a7eec6f,b27d2f78-2ed5-4831-8e41-fc8e31b02010,9530ce3f-c6fa-42f5-acf9-d11a82d1739c,...,[],"['Pharm Tech', 'Casuals', 'Cleaners', 'KRCHN',...","['GENERAL SUPPORT STAFFS', 'MEDICAL LABORATORY...",True,2024-08-01T10:04:31.201189Z,False,True,b1e558a5-3873-481c-bf23-9359e5d0232b,39.641468,-0.456223
3,31991.0,Jaystel Medical Centre(Kirinyaga),Jaystel Medical Centre(Kirinyaga),021244,Level 2,Medical Clinic,MEDICAL CLINIC,ecbf61a6-cd6d-4806-99d8-9340572c0015,5609901e-242d-49e0-b634-58ba485328db,01231885-b251-4349-8e12-cf15ee9dde03,...,[],"['Casuals', 'Cleaners', 'KECHN', 'Medical Labo...","['GENERAL SUPPORT STAFFS', 'MEDICAL LABORATORY...",True,2024-07-30T07:45:43.532577Z,False,True,68eb00e5-0f0c-4428-be96-57ec73301fee,37.294860,-0.663451
4,31989.0,Masimba Mission Hospital,St Francis Mission Hospital,11969,NaN,Basic Health Centre,HEALTH CENTRE,b6b5db70-609a-4194-888d-7841e02e9045,71918053-4a3f-4cba-86bc-d4e6c6a838be,4b9fa0df-c202-4a58-85e2-32873e1c1e29,...,[],"['Pharm Tech', 'Casuals', 'Clerks', 'Health Re...","['GENERAL SUPPORT STAFFS', 'MEDICAL LABORATORY...",True,2024-07-29T07:39:20.736265Z,False,True,3d65121b-c23f-402f-8b6f-db1e147ee690,34.941876,-0.874561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14800,12456.0,Makueni Catholic Dispensary,Makueni Catholic Dispensary,NaN,Level 2,Dispensary,DISPENSARY,ebf51b4b-50ad-4c3e-9cc3-fe04b7eb3b21,3b6d006a-f950-408b-8187-f4ccf82feb31,c84c128e-44b2-4c37-b992-c107c5474e01,...,[],[],[],True,2009-10-31T00:00:00Z,False,True,16282f87-3f90-4ffd-bb0a-ed5a5e4a44f7,37.624720,-1.804600
14801,14866.0,Kilibwoni Health Centre,Kilibwoni Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,ea02a4fd-f70f-4b8b-aff2-ae7c26b136fa,d501d0eb-e58a-4949-810d-a6397718d21a,2eaf9de2-ec59-4e1e-8c95-5f4bb913fd0d,...,[],[],[],True,2009-10-31T00:00:00Z,False,True,6ac6efda-63ec-46cd-a2b8-ad4391929cac,35.229760,0.208950
14802,16356.0,Mwangaza Medical Clinic,Mwangaza Medical Clinic,NaN,Level 2,Medical Clinic,MEDICAL CLINIC,1283e6ea-077d-4d32-8099-6d6acb428fd1,83e492c2-9431-482e-947e-48f387c73e70,51756633-f769-45d8-8684-793b05cc6b16,...,[],[],[],True,2009-10-31T00:00:00Z,False,True,281d6e79-31a3-485e-a823-b75d515b2190,35.250600,0.559882
14803,11171.0,Wanjengi Health Centre,Wanjengi Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,b9781c38-b053-466e-b95c-af288e27a0e6,2445b21b-966f-4e31-8827-54c474726607,2bb40b44-e8b1-494f-bc75-dc3530469a52,...,[],"['Community Health Extension/Assistants', 'Cle...","['MEDICAL LABORATORY', 'SUPPORT STAFF', 'CLINI...",True,2009-10-31T00:00:00Z,False,True,1f88e95f-be5b-4289-a4c8-7f8f844a26a6,37.011700,-0.724190


### Convert to gdf

In [24]:
# Create a GeoDataFrame by converting latitude and longitude to geometry points
geometry = [Point(xy) for xy in zip(df['lat'], df['long'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Set the coordinate reference system (CRS) to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

,code,name,officialname,registration_number,keph_level_name,facility_type_name,facility_type_category,county,constituency,ward,...,speciality_names,speciality_categories,is_public_visible,created,closed,is_published,id,lat,long,geometry
0,31996.0,Nyamira Teaching & Referral Hospital Private Wing,Nyamira Teaching and Referral Hospital Private...,NaN,Level 5,Secondary care hospitals,HOSPITALS,bbc8803a-7d6f-411a-96f3-5f8472b40405,b6d8c1ed-7411-4368-8ffe-36f506064458,39046919-0756-45c4-a84d-fdb584885ee5,...,"['Anaesthetist Nurse', 'Clerks', 'Human Resour...","['DENTAL STAFF', 'HEALTH ADMINISTRATIVE STAFFS...",True,2024-08-02T06:53:18.962471Z,False,True,accf47e7-2a44-4fd3-bbad-a711829e3927,34.929272,-0.557114,POINT (34.92927 -0.55711)
1,31999.0,Garissa Level 3 Hospital (Modika),Garissa Level 3 Hospital (Modika),NaN,Level 3,Comprehensive Health Centre,HEALTH CENTRE,4c6396a2-b23a-436a-a457-50e29a7eec6f,b27d2f78-2ed5-4831-8e41-fc8e31b02010,8f41801c-af0d-49e1-b519-79d0f9926326,...,"['General Medical Officers', 'Casuals', 'Pharm...","['DENTAL STAFF', 'MEDICAL OFFICERS & SPECIALIS...",True,2024-08-01T11:34:03.514866Z,False,True,55b23108-e2bd-4ceb-b58b-eb008a40ed0f,39.746100,-0.395990,POINT (39.74610 -0.39599)
2,31995.0,Equity Afia Medical Centre (Garissa),Equity Afia Medical Centre (Garissa),021173,Level 3,Medical Center,MEDICAL CENTER,4c6396a2-b23a-436a-a457-50e29a7eec6f,b27d2f78-2ed5-4831-8e41-fc8e31b02010,9530ce3f-c6fa-42f5-acf9-d11a82d1739c,...,"['Pharm Tech', 'Casuals', 'Cleaners', 'KRCHN',...","['GENERAL SUPPORT STAFFS', 'MEDICAL LABORATORY...",True,2024-08-01T10:04:31.201189Z,False,True,b1e558a5-3873-481c-bf23-9359e5d0232b,39.641468,-0.456223,POINT (39.64147 -0.45622)
3,31991.0,Jaystel Medical Centre(Kirinyaga),Jaystel Medical Centre(Kirinyaga),021244,Level 2,Medical Clinic,MEDICAL CLINIC,ecbf61a6-cd6d-4806-99d8-9340572c0015,5609901e-242d-49e0-b634-58ba485328db,01231885-b251-4349-8e12-cf15ee9dde03,...,"['Casuals', 'Cleaners', 'KECHN', 'Medical Labo...","['GENERAL SUPPORT STAFFS', 'MEDICAL LABORATORY...",True,2024-07-30T07:45:43.532577Z,False,True,68eb00e5-0f0c-4428-be96-57ec73301fee,37.294860,-0.663451,POINT (37.29486 -0.66345)
4,31989.0,Masimba Mission Hospital,St Francis Mission Hospital,11969,NaN,Basic Health Centre,HEALTH CENTRE,b6b5db70-609a-4194-888d-7841e02e9045,71918053-4a3f-4cba-86bc-d4e6c6a838be,4b9fa0df-c202-4a58-85e2-32873e1c1e29,...,"['Pharm Tech', 'Casuals', 'Clerks', 'Health Re...","['GENERAL SUPPORT STAFFS', 'MEDICAL LABORATORY...",True,2024-07-29T07:39:20.736265Z,False,True,3d65121b-c23f-402f-8b6f-db1e147ee690,34.941876,-0.874561,POINT (34.94188 -0.87456)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14800,12456.0,Makueni Catholic Dispensary,Makueni Catholic Dispensary,NaN,Level 2,Dispensary,DISPENSARY,ebf51b4b-50ad-4c3e-9cc3-fe04b7eb3b21,3b6d006a-f950-408b-8187-f4ccf82feb31,c84c128e-44b2-4c37-b992-c107c5474e01,...,[],[],True,2009-10-31T00:00:00Z,False,True,16282f87-3f90-4ffd-bb0a-ed5a5e4a44f7,37.624720,-1.804600,POINT (37.62472 -1.80460)
14801,14866.0,Kilibwoni Health Centre,Kilibwoni Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,ea02a4fd-f70f-4b8b-aff2-ae7c26b136fa,d501d0eb-e58a-4949-810d-a6397718d21a,2eaf9de2-ec59-4e1e-8c95-5f4bb913fd0d,...,[],[],True,2009-10-31T00:00:00Z,False,True,6ac6efda-63ec-46cd-a2b8-ad4391929cac,35.229760,0.208950,POINT (35.22976 0.20895)
14802,16356.0,Mwangaza Medical Clinic,Mwangaza Medical Clinic,NaN,Level 2,Medical Clinic,MEDICAL CLINIC,1283e6ea-077d-4d32-8099-6d6acb428fd1,83e492c2-9431-482e-947e-48f387c73e70,51756633-f769-45d8-8684-793b05cc6b16,...,[],[],True,2009-10-31T00:00:00Z,False,True,281d6e79-31a3-485e-a823-b75d515b2190,35.250600,0.559882,POINT (35.25060 0.55988)
14803,11171.0,Wanjengi Health Centre,Wanjengi Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,b9781c38-b053-466e-b95c-af288e27a0e6,2445b21b-966f-4e31-8827-54c474726607,2bb40b44-e8b1-494f-bc75-dc3530469a52,...,"['Community Health Extension/Assistants'

In [27]:
gdf.to_postgis(name='mhfl_ke', con=engine, if_exists='replace', index=False)


In [28]:
# Inspect the database
insp = inspect(engine)
tables = insp.get_table_names()
print(tables)  # Print the names of the tables

['spatial_ref_sys', 'pointcloud_formats', 'topology', 'layer', 'mhfl_ke']
